In [5]:
import numpy as np
import math
import random
import timeit
from scipy.special import expit

data = np.genfromtxt('diabetes.csv',delimiter=',')
for k in range(len(data)):
    if data[k][8]==0:
        data[k][8]=-1
        
data_x = data[1:500]
data_t = data[500:]

dtype_ = np.dtype(dtype="float64")

input_data=[]

for r in range(len(data_x)):
    input_data.append([data_x[r][:7],data_x[r][8:]])

testing_data=[]

for s in range(len(data_t)):
    testing_data.append([data_t[s][:7],data_t[s][8:]])

#Setting up the data type for numpy arrays
dtype="float64"
dtype_ = np.dtype(dtype)
x_i = list()
y_i = list()
for i, j in input_data:
        x_i.append(i)
        y_i.append(j)
x_i, y_i = np.array(x_i, dtype=dtype_), np.array(y_i, dtype=dtype_)

#Initializing the values of the Omega and Beta
start_data_w= np.random.randint(np.size(x_i[0]), size=np.size(x_i[0]))
start_data_b = np.random.randint(np.size(y_i))

#Gradient function
def gradient(training,W,B):
    L = 0.0001
    m = np.size(W)
    G0 = 2*L*W
    
    for x,y in input_data:
        #New array with the specified precision
        z = np.sum(-y*(np.sum(W*x[:, None]) + B))
        z = expit(z)
        sumgrad = np.subtract(np.sum(np.sum(-y*x)*np.exp(z)),np.sum(np.sum(y*np.exp(z))))/np.add(1,np.exp(np.sum(-y*np.exp(z))))
    return((1/m)*np.sum(np.add(G0,sumgrad)))

def sgm(gradient, training_d, start_w, start_b, stepsize, tolerance, n_iter, batch_size,
        dtype="float64", random_state=None):
    
    start = timeit.default_timer()
    
    #Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    #Setting up the data type for numpy arrays
    dtype_ = np.dtype(dtype)

    #Converting x and y to numpy arrays
    n_obs = x_i.shape[0]
    if n_obs != y_i.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    x_iy_i = np.c_[x_i.reshape(n_obs, -1), y_i.reshape(n_obs, 1)]

    #Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    #Initializing the values of the variables
    StartingV = [start_w, start_b]
    vector = np.array(start_w, dtype=dtype_)
    beta = np.array(start_b, dtype=dtype_)

    #Setting up and checking the stepsize
    stepsize = np.array(stepsize, dtype=dtype_)
    if np.any(stepsize <= 0):
        raise ValueError("'stepsize' must be greater than zero")

    #Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    #Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    #Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")
    
    #Performing the gradient descent loop
    for _ in range(n_iter):
        #Shuffle x and y
        rng.shuffle(x_iy_i)

        #Performing minibatch moves
        for StartingV in range(0, n_obs, batch_size):
            stop_t = StartingV + batch_size
            x_batch, y_batch = x_iy_i[StartingV:stop_t, :-1], x_iy_i[StartingV:stop_t, -1:]

            #Recalculating the difference
            diff = -stepsize*np.array(gradient([x_batch, y_batch], vector, beta), dtype_)

            #Checking if the absolute difference is small enough
            if np.all(np.abs(diff) > tolerance):
                #Updating the values of the variables
                vector -= diff
                beta -= diff
            else:
                print("break",start,stop)
                break
                
        print(vector,beta)   
    count_accuracy = 0
    
    for h,l in testing_data:
        if np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))>=0 and np.array(l)==1:
            count_accuracy += 1

        elif np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))<0 and np.array(l)==-1:
            count_accuracy += 1
            
        check_accuracy = np.divide(count_accuracy,len(testing_data))*100
        
    stop = timeit.default_timer()
    
    return(vector if vector.shape else vector.item(), stop-start,
           print("The local minimum occurs at: Omega={}".format(vector), "Beta={}".format(beta)),
          print("Reported accuracy:{}%".format(check_accuracy)), print('Run time is: ', stop-start))

print(sgm(gradient, input_data, start_data_w, start_data_b, 0.0001, 1e-04, 500, 1))

[-49.6167944 -51.6167944 -50.6167944 -48.6167944 -46.6167944 -52.6167944
 -46.6167944] 149.38320560033515
[-110.21995043 -112.21995043 -111.21995043 -109.21995043 -107.21995043
 -113.21995043 -107.21995043] 88.78004956548428
[-170.82371129 -172.82371129 -171.82371129 -169.82371129 -167.82371129
 -173.82371129 -167.82371129] 28.176288708124314
[-231.42807698 -233.42807698 -232.42807698 -230.42807698 -228.42807698
 -234.42807698 -228.42807698] -32.42807697778118
[-292.0330475 -294.0330475 -293.0330475 -291.0330475 -289.0330475
 -295.0330475 -289.0330475] -93.03304749826817
[-352.63862286 -354.63862286 -353.63862286 -351.63862286 -349.63862286
 -355.63862286 -349.63862286] -153.63862285937338
[-413.24480307 -415.24480307 -414.24480307 -412.24480307 -410.24480307
 -416.24480307 -410.24480307] -214.24480306713255
[-473.85158813 -475.85158813 -474.85158813 -472.85158813 -470.85158813
 -476.85158813 -470.85158813] -274.8515881275824
[-534.45897805 -536.45897805 -535.45897805 -533.45897805 -53